# Import libraries

In [207]:
import numpy as np
import pandas as pd
import statsmodels.stats as sm
from datetime import datetime
from datetime import timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.graph_objects as go

import os
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.2f}'.format)

# Download file

In [208]:
file_id = "1atOZX3YXmxx-_QebbTfndeC6U_DPTL2e" # ID of the file on Google Drive
file_name = 'Updated_data_2021&2022.csv'

%run download.ipynb

# Import data

In [209]:
# File path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, 'Data',file_name)

df = pd.read_csv(file_path)

df.head()

,DATE_DIM,DAY_OF_WEEK,BET_ACCOUNT_NUM_HASH,AGE,AGE_BAND,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,GROSS_MARGIN,TICKETS
0,2021-01-01,Fri,13154,67.00,65+,M,11846,WA,37.00,NaN,1081.00,NaN,1118.00,443.55,271.25,288
1,2021-01-01,Fri,18379,54.00,45-54,M,1884,WA,40.00,NaN,NaN,NaN,40.00,0.00,40.00,1
2,2021-01-01,Fri,559232,63.00,55-64,M,2866,WA,NaN,NaN,12.00,NaN,12.00,9.50,2.04,5
3,2021-01-01,Fri,698904,69.00,65+,M,2100,WA,NaN,NaN,1223.50,NaN,1223.50,267.91,245.12,40
4,2021-01-01,Fri,762921,67.00,65+,M,4766,WA,NaN,NaN,17.50,NaN,17.50,0.00,3.50,5


In [210]:
df.groupby('BET_ACCOUNT_NUM_HASH').size().sort_values(ascending=False)

BET_ACCOUNT_NUM_HASH
3636408132    730
3946065174    730
1086933289    729
581221986     729
671303409     729
             ... 
3016316450      1
266758072       1
3016351785      1
3016415898      1
2212042683      1
Length: 144869, dtype: int64

## Column type conversion

In [211]:
# DATE_DIM: datetime
df['DATE_DIM'] = pd.to_datetime(df['DATE_DIM'], format='%Y-%m-%d')

# BET_ACCOUNT_NUM_HASH: string
df['BET_ACCOUNT_NUM_HASH'] = df['BET_ACCOUNT_NUM_HASH'].astype('O')

## New columns

In [212]:
df['WIN_LOSS_RATIO'] = round(df['DIVIDENDS_PAID'] / df['TOTAL_TURNOVER'],2)
df['AVERAGE_TICKET_VALUE'] = df['TOTAL_TURNOVER'] / df['TICKETS']
df['WIN_LOSS'] = (df['WIN_LOSS_RATIO'] >= 1).astype('int')
df.index = df['DATE_DIM']

df = df.drop(['DATE_DIM', 'DAY_OF_WEEK', 'AGE_BAND', 'GROSS_MARGIN'], axis=1)
df.head()

,BET_ACCOUNT_NUM_HASH,AGE,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,TICKETS,WIN_LOSS_RATIO,AVERAGE_TICKET_VALUE,WIN_LOSS
DATE_DIM,,,,,,,,,,,,,,,
2021-01-01,13154,67.00,M,11846,WA,37.00,NaN,1081.00,NaN,1118.00,443.55,288,0.40,3.88,0
2021-01-01,18379,54.00,M,1884,WA,40.00,NaN,NaN,NaN,40.00,0.00,1,0.00,40.00,0
2021-01-01,559232,63.00,M,2866,WA,NaN,NaN,12.00,NaN,12.00,9.50,5,0.79,2.40,0
2021-01-01,698904,69.00,M,2100,WA,NaN,NaN,1223.50,NaN,1223.50,267.91,40,0.22,30.59,0
2021-01-01,762921,67.00,M,4766,WA,NaN,NaN,17.50,NaN,17.50,0.00,5,0.00,3.50,0


In [213]:
df.shape

(12364101, 15)

In [220]:
new_cus_df = df.groupby('BET_ACCOUNT_NUM_HASH').TENURE_IN_DAYS.min()
new_cus_df

BET_ACCOUNT_NUM_HASH
13154         11846
18379          1884
37251             0
129273            0
348318            0
              ...  
4294834703     4472
4294891076     2917
4294928673      728
4294964456        0
4294964854        3
Name: TENURE_IN_DAYS, Length: 144869, dtype: int64

In [221]:
new_cus = new_cus_df[new_cus_df==0]

In [223]:
df['WEEK'] = df.index.strftime('%U')
df['WEEK']

DATE_DIM
2021-01-01    00
2021-01-01    00
2021-01-01    00
2021-01-01    00
2021-01-01    00
              ..
2022-12-31    52
2022-12-31    52
2022-12-31    52
2022-12-31    52
2022-12-31    52
Name: WEEK, Length: 12364101, dtype: object

In [224]:
df['YEAR'] = df.index.year

In [229]:
df = df[df['BET_ACCOUNT_NUM_HASH'].isin(new_cus.index)]

In [230]:
df

,BET_ACCOUNT_NUM_HASH,AGE,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,TICKETS,WIN_LOSS_RATIO,AVERAGE_TICKET_VALUE,WIN_LOSS,WEEK,YEAR
DATE_DIM,,,,,,,,,,,,,,,,,
2021-01-01,47481981,30.00,U,0,WA,116.00,NaN,NaN,NaN,116.00,227.90,24,1.96,4.83,1,00,2021
2021-01-01,79827030,28.00,U,0,WA,40.00,NaN,90.00,NaN,130.00,55.00,8,0.42,16.25,0,00,2021
2021-01-01,81985845,62.00,U,0,OTH,20.00,NaN,NaN,NaN,20.00,0.00,1,0.00,20.00,0,00,2021
2021-01-01,98925704,44.00,U,0,WA,2.00,NaN,20.00,NaN,22.00,9.80,7,0.45,3.14,0,00,2021
2021-01-01,208524667,21.00,U,0,WA,27.00,NaN,6.00,NaN,33.00,40.15,8,1.22,4.12,1,00,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,4285468846,22.00,U,253,WA,NaN,30.00,NaN,NaN,30.00,23.20,2,0.77,15.00,0,52,2022
2022-12-31,4289621869,52.00,M,0,WA,90.00,NaN,12.00,NaN,102.00,245.00,13,2.40,7.85,1,52,2022
2022-12-31,4290236828,19.00,U,9,WA,25.00,NaN,NaN,NaN,25.00,0.00,4,0.00,6.25,0,52,2022


In [244]:
cus_df = df.reset_index().groupby(['BET_ACCOUNT_NUM_HASH', 'YEAR', 'WEEK'], as_index=False).agg({
    'DATE_DIM' : 'count',
    'AGE' : 'max',
    'GENDER' : 'max',
    'TENURE_IN_DAYS' : 'max',
    'RESIDENTIAL_STATE': 'max',
    'FOB_RACING_TURNOVER' : 'sum',
    'FOB_SPORT_TURNOVER' : 'sum',
    'PARI_RACING_TURNOVER' : 'sum', 
    'PARI_SPORT_TURNOVER' : 'sum', 
    'TOTAL_TURNOVER' : 'sum',
    'DIVIDENDS_PAID' : 'sum', 
    'TICKETS' : 'sum',
    'WIN_LOSS' : 'sum'
})

In [245]:
cus_df

,BET_ACCOUNT_NUM_HASH,YEAR,WEEK,DATE_DIM,AGE,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,TICKETS,WIN_LOSS
0,37251,2022,44,1,22.00,U,0,WA,65.00,0.00,35.00,0.00,100.00,760.00,7,1
1,129273,2021,52,1,22.00,U,0,WA,20.00,0.00,0.00,0.00,20.00,0.00,9,0
2,129273,2022,03,1,22.00,U,27,WA,25.00,0.00,0.00,0.00,25.00,0.00,10,0
3,129273,2022,27,2,22.00,U,195,WA,0.00,25.00,0.00,0.00,25.00,14.41,3,0
4,129273,2022,28,4,22.00,U,202,WA,0.00,26.00,0.00,0.00,26.00,21.95,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257675,4294749281,2022,25,1,33.00,U,330,WA,0.00,0.00,143.00,0.00,143.00,43.18,71,0
257676,4294749281,2022,30,1,33.00,U,367,WA,0.00,0.00,100.00,0.00,100.00,0.00,62,0
257677,4294749281,2022,31,1,33.00,U,377,WA,0.00,0.00,195.00,0.00,195.00,0.00,55,0
257678,4294749281,2022,33,1,33.00,U,385,WA,20.00,0.00,180.00,0.00,200.00,0.00,41,0


In [248]:
today = datetime(2022,12,31)
today

datetime.datetime(2022, 12, 31, 0, 0)

In [249]:
def aggregate_by_period(series, func=np.size , from_date = datetime(2022,12,31), days=30):
    start_date = from_date - timedelta(days=days-1)
    filtered_df = series[start_date:from_date]
    result_df = func(filtered_df)
    
    return result_df

In [250]:
div_sum_func_list = [
    lambda x: aggregate_by_period(x,func=np.sum,days=30),
    lambda x: aggregate_by_period(x,func=np.sum,days=90),
    lambda x: aggregate_by_period(x,func=np.sum,days=180)
]

turnover_sum_func_list = [
    lambda x: aggregate_by_period(x,func=np.sum,days=30),
    lambda x: aggregate_by_period(x,func=np.sum,days=90),
    lambda x: aggregate_by_period(x,func=np.sum,days=180)
]

count_func_list = [
    lambda x: aggregate_by_period(x,func=np.size,days=30),
    lambda x: aggregate_by_period(x,func=np.size,days=90),
    lambda x: aggregate_by_period(x,func=np.size,days=180)
]

In [251]:
cus_df1 = df.groupby('BET_ACCOUNT_NUM_HASH').agg({
    'DIVIDENDS_PAID': div_sum_func_list,
    'BET_ACCOUNT_NUM_HASH': count_func_list,
    'TOTAL_TURNOVER' : turnover_sum_func_list
})

In [252]:
cus_df1.columns = ['DIVIDENDS_PAID_30',
                   'DIVIDENDS_PAID_90',
                   'DIVIDENDS_PAID_180',
                   'BET_ACCOUNT_NUM_HASH_30',
                   'BET_ACCOUNT_NUM_HASH_90',
                   'BET_ACCOUNT_NUM_HASH_180',
                   'TOTAL_TURNOVER_30',
                   'TOTAL_TURNOVER_90',
                   'TOTAL_TURNOVER_180'
                  ]

,DIVIDENDS_PAID_30,DIVIDENDS_PAID_90,DIVIDENDS_PAID_180,BET_ACCOUNT_NUM_HASH_30,BET_ACCOUNT_NUM_HASH_90,BET_ACCOUNT_NUM_HASH_180,TOTAL_TURNOVER_30,TOTAL_TURNOVER_90,TOTAL_TURNOVER_180
BET_ACCOUNT_NUM_HASH,,,,,,,,,
37251,0.00,760.00,760.00,0,1,1,0.00,100.00,100.00
129273,32.50,111.25,149.45,3,5,14,15.00,42.00,110.00
348318,0.00,0.00,0.00,0,0,3,0.00,0.00,20.00
368326,0.00,59.80,59.80,0,1,1,0.00,87.50,87.50
575313,0.00,43.77,43.77,3,5,5,40.00,63.97,63.97
...,...,...,...,...,...,...,...,...,...
4293672442,0.00,36.00,36.00,0,1,1,0.00,20.00,20.00
4293920247,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00
4294296954,4275.50,4275.50,4275.50,8,8,8,3500.00,3500.00,3500.00


In [253]:
cus_df1['BET_ACCOUNT_NUM_HASH_180']

BET_ACCOUNT_NUM_HASH
37251          1
129273        14
348318         3
368326         1
575313         5
              ..
4293672442     1
4293920247     0
4294296954     8
4294749281     3
4294964456     1
Name: BET_ACCOUNT_NUM_HASH_180, Length: 29176, dtype: int64

In [254]:
cus_df1['WIN_LOSS_RATIO_30'] = cus_df1['DIVIDENDS_PAID_30'] / cus_df1['TOTAL_TURNOVER_30']
cus_df1['WIN_LOSS_RATIO_90'] = cus_df1['DIVIDENDS_PAID_90'] / cus_df1['TOTAL_TURNOVER_90']
cus_df1['WIN_LOSS_RATIO_180'] = cus_df1['DIVIDENDS_PAID_180'] / cus_df1['TOTAL_TURNOVER_180']

In [255]:
cus_df1

,DIVIDENDS_PAID_30,DIVIDENDS_PAID_90,DIVIDENDS_PAID_180,BET_ACCOUNT_NUM_HASH_30,BET_ACCOUNT_NUM_HASH_90,BET_ACCOUNT_NUM_HASH_180,TOTAL_TURNOVER_30,TOTAL_TURNOVER_90,TOTAL_TURNOVER_180,WIN_LOSS_RATIO_30,WIN_LOSS_RATIO_90,WIN_LOSS_RATIO_180
BET_ACCOUNT_NUM_HASH,,,,,,,,,,,,
37251,0.00,760.00,760.00,0,1,1,0.00,100.00,100.00,NaN,7.60,7.60
129273,32.50,111.25,149.45,3,5,14,15.00,42.00,110.00,2.17,2.65,1.36
348318,0.00,0.00,0.00,0,0,3,0.00,0.00,20.00,NaN,NaN,0.00
368326,0.00,59.80,59.80,0,1,1,0.00,87.50,87.50,NaN,0.68,0.68
575313,0.00,43.77,43.77,3,5,5,40.00,63.97,63.97,0.00,0.68,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...
4293672442,0.00,36.00,36.00,0,1,1,0.00,20.00,20.00,NaN,1.80,1.80
4293920247,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,NaN,NaN,NaN
4294296954,4275.50,4275.50,4275.50,8,8,8,3500.00,3500.00,3500.00,1.22,1.22,1.22


In [256]:
cus_df1.sort_values(['BET_ACCOUNT_NUM_HASH_30', 'BET_ACCOUNT_NUM_HASH_90', 'BET_ACCOUNT_NUM_HASH_180'],ascending=False)

,DIVIDENDS_PAID_30,DIVIDENDS_PAID_90,DIVIDENDS_PAID_180,BET_ACCOUNT_NUM_HASH_30,BET_ACCOUNT_NUM_HASH_90,BET_ACCOUNT_NUM_HASH_180,TOTAL_TURNOVER_30,TOTAL_TURNOVER_90,TOTAL_TURNOVER_180,WIN_LOSS_RATIO_30,WIN_LOSS_RATIO_90,WIN_LOSS_RATIO_180
BET_ACCOUNT_NUM_HASH,,,,,,,,,,,,
1265470183,10130.00,37687.46,68098.98,30,90,180,12522.86,41639.90,79062.11,0.81,0.91,0.86
2126133124,5189.81,13026.02,24228.94,30,89,178,5705.46,14081.91,26039.70,0.91,0.93,0.93
4269980491,6745.71,21156.01,31736.58,30,89,159,6246.61,20157.79,30008.16,1.08,1.05,1.06
916834555,28772.70,75350.71,102335.31,30,87,158,30806.80,81814.46,112955.96,0.93,0.92,0.91
1468198948,4399.79,7184.96,21968.45,30,83,151,4046.79,9758.96,23778.45,1.09,0.74,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...
4292485120,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,NaN,NaN,NaN
4293088138,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,NaN,NaN,NaN
4293108116,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,NaN,NaN,NaN


In [ ]:
def cusum(data, c=0):
    m = np.mean(data)
    s_list = [0]
    
    for i in data[1:]:
        s = max(0, i + s_list[-1] - m + c)
        s_list.append(s)
        
    return s_list

In [ ]:
plt.plot(cusum(turnover))